In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [2]:
import polars as pl
import polars.selectors as cs
import yaml

from utils.preprocessing import *
from config.constants import BATCH_SIZE

/Users/tung/Repos/cxc2025-federato/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data Preprocessing

We will proceed to process the concatenated dataset using the `preprocessing` module which we wrote.

This module contains some of the useful functions and configs we could use to process the data.

### Read YAML file

We will also read our preprocessing instructions from our config file, which is saved in `preprocessing.yaml`

In [3]:
# Read YAML file
config_file_path = '../config/preprocessing.yaml'
with open(config_file_path) as fstream:
    config = yaml.safe_load(fstream)

### Import Dataset

We will first import the dataset into our notebook

In [ ]:
# Process data
file_path = os.path.expanduser(config['pipeline']['input_data'])
df = pl.read_parquet(file_path)
df

### Drop Columns

First, we will drop some of the columns that we have decided to not use for modelling during the EDA phase.

In [ ]:
df = drop_columns(df, config, 0)
df

In [ ]:
# Expand dict columns
df = batch_preprocess_data(df, expand_dict_columns, config, BATCH_SIZE)
df

### Enforce Types

Next, we will convert the types of each column to our desired type

In [5]:
# Enforce Types
df = custom_enforce_types(df, config)
df

$insert_id,amplitude_id,app,city,client_event_time,client_upload_time,country,device_family,device_id,dma,event_id,event_properties,event_time,event_type,language,library,os_name,processed_time,region,server_received_time,server_upload_time,session_id,user_id,user_properties,roles,isInternalUser,referrer,slug
str,i64,i64,str,datetime[μs],datetime[μs],str,str,str,str,i64,str,datetime[μs],str,str,str,str,datetime[μs],str,datetime[μs],datetime[μs],i64,str,str,str,str,str,str
"""251db963-6623-448e-8665-f542b8…",935023330069,591532,"""Mumbai""",2024-10-08 11:41:02.385,2024-10-08 11:41:04.857,"""India""","""Windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…","""None""",22216,"""{}""",2024-10-08 11:41:02.385,"""session_start""","""English""","""amplitude-ts/2.7.2""","""Chrome""",2024-10-08 11:41:05.371,"""Maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662385,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""False""","""https://accounts.google.com/""",null
"""3a95cdac-174c-4002-8e84-8aeba8…",935023330069,591532,"""Mumbai""",2024-10-08 11:41:02.386,2024-10-08 11:41:04.857,"""India""","""Windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…","""None""",22215,"""{}""",2024-10-08 11:41:02.386,"""session_end""","""English""","""amplitude-ts/2.7.2""","""Chrome""",2024-10-08 11:41:05.371,"""Maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662385,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""False""","""https://accounts.google.com/""",null
"""50dbfad3-8e24-456a-ae68-21b09b…",935023330069,591532,"""Mumbai""",2024-10-08 11:41:02.386,2024-10-08 11:41:04.857,"""India""","""Windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…","""None""",22216,"""{'[Amplitude] Session Replay I…",2024-10-08 11:41:02.386,"""session_start""","""English""","""amplitude-ts/2.7.2""","""Chrome""",2024-10-08 11:41:05.371,"""Maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""False""","""https://accounts.google.com/""",null
"""ea0a4143-2380-48e7-a83e-8ba854…",935023330069,591532,"""Mumbai""",2024-10-08 11:41:03.500,2024-10-08 11:41:04.857,"""India""","""Windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…","""None""",22217,"""{'rowModel': 'server', '[Ampli…",2024-10-08 11:41:03.500,""":all-accounts:configurable-tab…","""English""","""amplitude-ts/2.7.2""","""Chrome""",2024-10-08 11:41:05.371,"""Maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""False""","""https://accounts.google.com/""",null
"""4d89977d-4734-450c-afa1-07e326…",935023330069,591532,"""Mumbai""",2024-10-08 11:41:03.527,2024-10-08 11:41:04.857,"""India""","""Windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…","""None""",22217,"""{'displayName': 'All Policies …",2024-10-08 11:41:03.527,""":all-accounts:widget:render""","""English""","""amplitude-ts/2.7.2""","""Chrome""",2024-10-08 11:41:05.371,"""Maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""False""","""https://accounts.google.com/""",null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""102ec3ca-a467-41a9-aa19-94f5b0…",857540480084,591532,"""Nashville""",2024-05-22 09:36:09.888,2024-05-22 09:36:10.832,"""United States""","""Windows""","""a8b357fb-b4fb-45d2-babd-42b470…","""Nashville, TN""",26,"""{'displayName': 'One Drive Lin…",2024-05-22 09:36:09.888,"""account-lines::widget:render""","""English""","""amplitude-ts/1.8.0""","""Edge""",2024-05-22 09:36:11.662,"""Tennessee""",2024-05-22 09:36:10.832,2024-05-22 09:36:10.833,1716368557821,"""9aecc15d-64a4-4190-80f5-b5b842…","""{'initial_utm_medium': 'EMPTY'…","""underwriter""","""False""",null,"""one-drive-link"""
"""55aec1ce-b876-4f66-b786-fd5706…"

### Lowercase All Values

In [6]:
# Lowercase all values
df = lowercase_all_values(df, config)
df

$insert_id,amplitude_id,app,city,client_event_time,client_upload_time,country,device_family,device_id,dma,event_id,event_properties,event_time,event_type,language,library,os_name,processed_time,region,server_received_time,server_upload_time,session_id,user_id,user_properties,roles,isInternalUser,referrer,slug
str,i64,i64,str,datetime[μs],datetime[μs],str,str,str,str,i64,str,datetime[μs],str,str,str,str,datetime[μs],str,datetime[μs],datetime[μs],i64,str,str,str,str,str,str
"""251db963-6623-448e-8665-f542b8…",935023330069,591532,"""mumbai""",2024-10-08 11:41:02.385,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…","""none""",22216,"""{}""",2024-10-08 11:41:02.385,"""session_start""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662385,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null
"""3a95cdac-174c-4002-8e84-8aeba8…",935023330069,591532,"""mumbai""",2024-10-08 11:41:02.386,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…","""none""",22215,"""{}""",2024-10-08 11:41:02.386,"""session_end""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662385,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null
"""50dbfad3-8e24-456a-ae68-21b09b…",935023330069,591532,"""mumbai""",2024-10-08 11:41:02.386,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…","""none""",22216,"""{'[amplitude] session replay i…",2024-10-08 11:41:02.386,"""session_start""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null
"""ea0a4143-2380-48e7-a83e-8ba854…",935023330069,591532,"""mumbai""",2024-10-08 11:41:03.500,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…","""none""",22217,"""{'rowmodel': 'server', '[ampli…",2024-10-08 11:41:03.500,""":all-accounts:configurable-tab…","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null
"""4d89977d-4734-450c-afa1-07e326…",935023330069,591532,"""mumbai""",2024-10-08 11:41:03.527,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…","""none""",22217,"""{'displayname': 'all policies …",2024-10-08 11:41:03.527,""":all-accounts:widget:render""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""102ec3ca-a467-41a9-aa19-94f5b0…",857540480084,591532,"""nashville""",2024-05-22 09:36:09.888,2024-05-22 09:36:10.832,"""united states""","""windows""","""a8b357fb-b4fb-45d2-babd-42b470…","""nashville, tn""",26,"""{'displayname': 'one drive lin…",2024-05-22 09:36:09.888,"""account-lines::widget:render""","""english""","""amplitude-ts/1.8.0""","""edge""",2024-05-22 09:36:11.662,"""tennessee""",2024-05-22 09:36:10.832,2024-05-22 09:36:10.833,1716368557821,"""9aecc15d-64a4-4190-80f5-b5b842…","""{'initial_utm_medium': 'empty'…","""underwriter""","""false""",null,"""one-drive-link"""
"""55aec1ce-b876-4f66-b786-fd5706…"

### Replace Null Representations

We will also replace some commmon null representations like `empty` and `none` to null values

In [7]:
# Replace with null
df = replace_with_null(df, config)
df

$insert_id,amplitude_id,app,city,client_event_time,client_upload_time,country,device_family,device_id,dma,event_id,event_properties,event_time,event_type,language,library,os_name,processed_time,region,server_received_time,server_upload_time,session_id,user_id,user_properties,roles,isInternalUser,referrer,slug
str,i64,i64,str,datetime[μs],datetime[μs],str,str,str,str,i64,str,datetime[μs],str,str,str,str,datetime[μs],str,datetime[μs],datetime[μs],i64,str,str,str,str,str,str
"""251db963-6623-448e-8665-f542b8…",935023330069,591532,"""mumbai""",2024-10-08 11:41:02.385,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22216,"""{}""",2024-10-08 11:41:02.385,"""session_start""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662385,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null
"""3a95cdac-174c-4002-8e84-8aeba8…",935023330069,591532,"""mumbai""",2024-10-08 11:41:02.386,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22215,"""{}""",2024-10-08 11:41:02.386,"""session_end""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662385,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null
"""50dbfad3-8e24-456a-ae68-21b09b…",935023330069,591532,"""mumbai""",2024-10-08 11:41:02.386,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22216,"""{'[amplitude] session replay i…",2024-10-08 11:41:02.386,"""session_start""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null
"""ea0a4143-2380-48e7-a83e-8ba854…",935023330069,591532,"""mumbai""",2024-10-08 11:41:03.500,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22217,"""{'rowmodel': 'server', '[ampli…",2024-10-08 11:41:03.500,""":all-accounts:configurable-tab…","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null
"""4d89977d-4734-450c-afa1-07e326…",935023330069,591532,"""mumbai""",2024-10-08 11:41:03.527,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22217,"""{'displayname': 'all policies …",2024-10-08 11:41:03.527,""":all-accounts:widget:render""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""102ec3ca-a467-41a9-aa19-94f5b0…",857540480084,591532,"""nashville""",2024-05-22 09:36:09.888,2024-05-22 09:36:10.832,"""united states""","""windows""","""a8b357fb-b4fb-45d2-babd-42b470…","""nashville, tn""",26,"""{'displayname': 'one drive lin…",2024-05-22 09:36:09.888,"""account-lines::widget:render""","""english""","""amplitude-ts/1.8.0""","""edge""",2024-05-22 09:36:11.662,"""tennessee""",2024-05-22 09:36:10.832,2024-05-22 09:36:10.833,1716368557821,"""9aecc15d-64a4-4190-80f5-b5b842…","""{'initial_utm_medium': 'empty'…","""underwriter""","""false""",null,"""one-drive-link"""
"""55aec1ce-b876-4f66-b786-fd5706…",857540480084,591532,"""nashvi

### Create Session Duration 

In [8]:
# Filter out empty user ids
df = df.filter(pl.col('user_id').is_not_null())
df

# Create session durations
user_session = (df.group_by(['user_id', 'session_id'])
                  .agg([pl.max('client_event_time').name.prefix('max_'),
                        pl.min('client_event_time').name.prefix('min_')]))

# Session time calculation 
user_session = user_session.with_columns((pl.col('max_client_event_time') 
                                          - pl.col('min_client_event_time')).alias('session_duration'))
user_session = user_session.with_columns(pl.col('session_duration').dt.total_seconds().alias('session_seconds'))


# Merge columns to original df
df = df.join(user_session.select('user_id', 'session_id', 'session_seconds'), on=['user_id', 'session_id'], how='left')
df

$insert_id,amplitude_id,app,city,client_event_time,client_upload_time,country,device_family,device_id,dma,event_id,event_properties,event_time,event_type,language,library,os_name,processed_time,region,server_received_time,server_upload_time,session_id,user_id,user_properties,roles,isInternalUser,referrer,slug,session_seconds
str,i64,i64,str,datetime[μs],datetime[μs],str,str,str,str,i64,str,datetime[μs],str,str,str,str,datetime[μs],str,datetime[μs],datetime[μs],i64,str,str,str,str,str,str,i64
"""251db963-6623-448e-8665-f542b8…",935023330069,591532,"""mumbai""",2024-10-08 11:41:02.385,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22216,"""{}""",2024-10-08 11:41:02.385,"""session_start""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662385,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null,0
"""3a95cdac-174c-4002-8e84-8aeba8…",935023330069,591532,"""mumbai""",2024-10-08 11:41:02.386,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22215,"""{}""",2024-10-08 11:41:02.386,"""session_end""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662385,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null,0
"""50dbfad3-8e24-456a-ae68-21b09b…",935023330069,591532,"""mumbai""",2024-10-08 11:41:02.386,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22216,"""{'[amplitude] session replay i…",2024-10-08 11:41:02.386,"""session_start""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null,2455
"""ea0a4143-2380-48e7-a83e-8ba854…",935023330069,591532,"""mumbai""",2024-10-08 11:41:03.500,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22217,"""{'rowmodel': 'server', '[ampli…",2024-10-08 11:41:03.500,""":all-accounts:configurable-tab…","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null,2455
"""4d89977d-4734-450c-afa1-07e326…",935023330069,591532,"""mumbai""",2024-10-08 11:41:03.527,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22217,"""{'displayname': 'all policies …",2024-10-08 11:41:03.527,""":all-accounts:widget:render""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null,2455
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""102ec3ca-a467-41a9-aa19-94f5b0…",857540480084,591532,"""nashville""",2024-05-22 09:36:09.888,2024-05-22 09:36:10.832,"""united states""","""windows""","""a8b357fb-b4fb-45d2-babd-42b470…","""nashville, tn""",26,"""{'displayname': 'one drive lin…",2024-05-22 09:36:09.888,"""account-lines::widget:render""","""english""","""amplitude-ts/1.8.0""","""edge""",2024-05-22 09:36:11.662,"""tennessee""",2024-05-22 09:36:10.832,2024-05-22 09:36:10.833,1716368557821,"""9aecc15d-64a4-4190-80f5-b5b842…","""{'initial_utm_medium': 'empty'…","""underwriter""","""false""",null,"""one-drive-link""",2371
"""55aec1ce-b876-4f

## Feature Engineering

Next, we will add some features which will serve to be useful in the modelling phase

### Group Regions 

We notice in EDA that most regions come from the United States, with a smaller subset in other countries.

For regions in the US, we will group them to four main regions: `northeast`, `midwest`, `south`, `west`

For any regions outside the US, we will classify them as `international`

In [9]:
def group_region_categories(df, input_col, output_col, condense_map_dict):

     # Get dictionary mapping from condense map
    map_dict = {}
    else_val = condense_map_dict.get('else', None)

    for key, val in condense_map_dict.items():
        if key == 'else':
            continue
        map_dict.update({itm : key for itm in val})
    
    print(map_dict)
    # Update values in the column with respective mappings
    df = df.with_columns(pl.col(input_col).replace(map_dict).alias(output_col))

    if else_val is not None:
        df = df.with_columns(pl.when(pl.col(output_col).is_in(condense_map_dict.keys()))
                                .then(pl.col(output_col))
                                .otherwise(pl.lit(else_val))
                                .alias(output_col))
    return df

In [10]:
# Specify region mappings
us_region_mapping = {
    'northeast': ['connecticut', 'maine', 'massachusetts', 'new hampshire', 'new jersey', 'new york', 'pennsylvania', 'rhode island', 'vermont'],
    'midwest': ['illinois', 'indiana', 'iowa', 'kansas', 'michigan', 'minnesota', 'missouri', 'nebraska', 'north dakota', 'ohio', 'south dakota', 'wisconsin'],
    'south': ['alabama', 'arkansas', 'delaware', 'florida', 'georgia', 'kentucky', 'louisiana', 'maryland', 'mississippi', 'north carolina', 'oklahoma', 
              'south carolina', 'tennessee', 'texas', 'virginia', 'west virginia'],
    'west': ['alaska', 'arizona', 'california', 'colorado', 'hawaii', 'idaho', 'montana', 'nevada', 'new mexico', 'oregon', 'utah', 'washington', 'wyoming'],
    'else': 'international'
}


In [11]:
# Encode regions
df = group_region_categories(df, 'region', 'region_grouped', us_region_mapping)
df

{'connecticut': 'northeast', 'maine': 'northeast', 'massachusetts': 'northeast', 'new hampshire': 'northeast', 'new jersey': 'northeast', 'new york': 'northeast', 'pennsylvania': 'northeast', 'rhode island': 'northeast', 'vermont': 'northeast', 'illinois': 'midwest', 'indiana': 'midwest', 'iowa': 'midwest', 'kansas': 'midwest', 'michigan': 'midwest', 'minnesota': 'midwest', 'missouri': 'midwest', 'nebraska': 'midwest', 'north dakota': 'midwest', 'ohio': 'midwest', 'south dakota': 'midwest', 'wisconsin': 'midwest', 'alabama': 'south', 'arkansas': 'south', 'delaware': 'south', 'florida': 'south', 'georgia': 'south', 'kentucky': 'south', 'louisiana': 'south', 'maryland': 'south', 'mississippi': 'south', 'north carolina': 'south', 'oklahoma': 'south', 'south carolina': 'south', 'tennessee': 'south', 'texas': 'south', 'virginia': 'south', 'west virginia': 'south', 'alaska': 'west', 'arizona': 'west', 'california': 'west', 'colorado': 'west', 'hawaii': 'west', 'idaho': 'west', 'montana': 'we

$insert_id,amplitude_id,app,city,client_event_time,client_upload_time,country,device_family,device_id,dma,event_id,event_properties,event_time,event_type,language,library,os_name,processed_time,region,server_received_time,server_upload_time,session_id,user_id,user_properties,roles,isInternalUser,referrer,slug,session_seconds,region_grouped
str,i64,i64,str,datetime[μs],datetime[μs],str,str,str,str,i64,str,datetime[μs],str,str,str,str,datetime[μs],str,datetime[μs],datetime[μs],i64,str,str,str,str,str,str,i64,str
"""251db963-6623-448e-8665-f542b8…",935023330069,591532,"""mumbai""",2024-10-08 11:41:02.385,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22216,"""{}""",2024-10-08 11:41:02.385,"""session_start""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662385,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null,0,"""international"""
"""3a95cdac-174c-4002-8e84-8aeba8…",935023330069,591532,"""mumbai""",2024-10-08 11:41:02.386,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22215,"""{}""",2024-10-08 11:41:02.386,"""session_end""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662385,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null,0,"""international"""
"""50dbfad3-8e24-456a-ae68-21b09b…",935023330069,591532,"""mumbai""",2024-10-08 11:41:02.386,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22216,"""{'[amplitude] session replay i…",2024-10-08 11:41:02.386,"""session_start""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null,2455,"""international"""
"""ea0a4143-2380-48e7-a83e-8ba854…",935023330069,591532,"""mumbai""",2024-10-08 11:41:03.500,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22217,"""{'rowmodel': 'server', '[ampli…",2024-10-08 11:41:03.500,""":all-accounts:configurable-tab…","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null,2455,"""international"""
"""4d89977d-4734-450c-afa1-07e326…",935023330069,591532,"""mumbai""",2024-10-08 11:41:03.527,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22217,"""{'displayname': 'all policies …",2024-10-08 11:41:03.527,""":all-accounts:widget:render""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null,2455,"""international"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""102ec3ca-a467-41a9-aa19-94f5b0…",857540480084,591532,"""nashville""",2024-05-22 09:36:09.888,2024-05-22 09:36:10.832,"""united states""","""windows""","""a8b357fb-b4fb-45d2-babd-42b470…","""nashville, tn""",26,"""{'displayname': 'one drive lin…",2024-05-22 09:36:09.888,"""account-lines::widget:render""","""english""","""amplitude-ts/1.8.0""","""edge""",2024-05-22 09:36:11.662,"""tennessee""",2024-05-22 09:36:10.832,2024-05-22 09:36:10.833,1716368557821,"""9aecc15d-64a4-4190-80f5-b5b8

### Encode Events

Moreover, we will also encode the events into 7 main categories:

- Session & Navigation
- Account & Policy Management
- Dashboard & UI Interactions
- Action Center & Workflow
- Submission & Forms
- Filtering & Searching
- Document & Report Interactions
- Other/System Events

All of the contents of this encoding has been written in the config, and can be applied by using `map_values` 

In [12]:
# Function for grouping the events
def categorize_event(event_type):
    event_type = event_type.lower()  # Ensure input is lowercase
    
    if any(keyword in event_type for keyword in ["session_start", "session_end", "application-window", "nav-header", "dashboard"]):
        return "session & navigation"
    elif any(keyword in event_type for keyword in ["account", "policy", "rating"]):
        return "account & policy management"
    elif any(keyword in event_type for keyword in ["dashboard", "widget", "layout", "insights", "table"]):
        return "dashboard & ui interactions"
    elif any(keyword in event_type for keyword in ["action-center", "task", "workflow", "take-action"]):
        return "action center & workflow"
    elif any(keyword in event_type for keyword in ["submit-click", "form", "create", "definition", "save-click", "submissions"]):
        return "submission & forms"
    elif any(keyword in event_type for keyword in ["filter", "sort", "search", "advanced-filters"]):
        return "filtering & searching"
    elif any(keyword in event_type for keyword in ["document", "report", "download", "csv"]):
        return "document & report interactions"
    else:
        return "other/system events"


In [13]:
# Encode events
df = df.with_columns(pl.col('event_type')
                     .map_elements(categorize_event, return_dtype = pl.String)
                     .alias('event_category'))
df

$insert_id,amplitude_id,app,city,client_event_time,client_upload_time,country,device_family,device_id,dma,event_id,event_properties,event_time,event_type,language,library,os_name,processed_time,region,server_received_time,server_upload_time,session_id,user_id,user_properties,roles,isInternalUser,referrer,slug,session_seconds,region_grouped,event_category
str,i64,i64,str,datetime[μs],datetime[μs],str,str,str,str,i64,str,datetime[μs],str,str,str,str,datetime[μs],str,datetime[μs],datetime[μs],i64,str,str,str,str,str,str,i64,str,str
"""251db963-6623-448e-8665-f542b8…",935023330069,591532,"""mumbai""",2024-10-08 11:41:02.385,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22216,"""{}""",2024-10-08 11:41:02.385,"""session_start""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662385,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null,0,"""international""","""session & navigation"""
"""3a95cdac-174c-4002-8e84-8aeba8…",935023330069,591532,"""mumbai""",2024-10-08 11:41:02.386,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22215,"""{}""",2024-10-08 11:41:02.386,"""session_end""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662385,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null,0,"""international""","""session & navigation"""
"""50dbfad3-8e24-456a-ae68-21b09b…",935023330069,591532,"""mumbai""",2024-10-08 11:41:02.386,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22216,"""{'[amplitude] session replay i…",2024-10-08 11:41:02.386,"""session_start""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null,2455,"""international""","""session & navigation"""
"""ea0a4143-2380-48e7-a83e-8ba854…",935023330069,591532,"""mumbai""",2024-10-08 11:41:03.500,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22217,"""{'rowmodel': 'server', '[ampli…",2024-10-08 11:41:03.500,""":all-accounts:configurable-tab…","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null,2455,"""international""","""account & policy management"""
"""4d89977d-4734-450c-afa1-07e326…",935023330069,591532,"""mumbai""",2024-10-08 11:41:03.527,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22217,"""{'displayname': 'all policies …",2024-10-08 11:41:03.527,""":all-accounts:widget:render""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null,2455,"""international""","""account & policy management"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""102ec3ca-a467-41a9-aa19-94f5b0…",857540480084,591532,"""nashville""",2024-05-22 09:36:09.888,2024-05-22 09:36:10.832,"""united states""","""windows""","""a8b357fb-b4fb-45d2-babd-42b470…","""nashville, tn""",26,"""{'displayname': 'one drive lin…",2024-05-22 09:36:09.888,"""account-lines::widget:render""","""english"""

### Datetime Feature Engineering

There is a lot of potential for feature engineering for the datetime columns.

We will split these datetime columns into their own components, while also calculating the relative time to indicate a notion of distance 

In [14]:
# Extract hours from datetime columns
datetime_cols = ["client_event_time", "client_upload_time", "event_time", 
                "processed_time", "server_received_time", "server_upload_time"]

time_to_hour_map = {
    'client_event_time': 'client_event_hour',
    'client_upload_time': 'client_upload_hour',
    'event_time': 'event_hour',
    'processed_time': 'processed_hour',
    'server_received_time': 'server_received_hour',
    'server_upload_time': 'server_upload_hour'
}

df = df.with_columns(pl.col(c).dt.hour().alias(time_to_hour_map[c]) for c in datetime_cols)
df

$insert_id,amplitude_id,app,city,client_event_time,client_upload_time,country,device_family,device_id,dma,event_id,event_properties,event_time,event_type,language,library,os_name,processed_time,region,server_received_time,server_upload_time,session_id,user_id,user_properties,roles,isInternalUser,referrer,slug,session_seconds,region_grouped,event_category,client_event_hour,client_upload_hour,event_hour,processed_hour,server_received_hour,server_upload_hour
str,i64,i64,str,datetime[μs],datetime[μs],str,str,str,str,i64,str,datetime[μs],str,str,str,str,datetime[μs],str,datetime[μs],datetime[μs],i64,str,str,str,str,str,str,i64,str,str,i8,i8,i8,i8,i8,i8
"""251db963-6623-448e-8665-f542b8…",935023330069,591532,"""mumbai""",2024-10-08 11:41:02.385,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22216,"""{}""",2024-10-08 11:41:02.385,"""session_start""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662385,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null,0,"""international""","""session & navigation""",11,11,11,11,11,11
"""3a95cdac-174c-4002-8e84-8aeba8…",935023330069,591532,"""mumbai""",2024-10-08 11:41:02.386,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22215,"""{}""",2024-10-08 11:41:02.386,"""session_end""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662385,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null,0,"""international""","""session & navigation""",11,11,11,11,11,11
"""50dbfad3-8e24-456a-ae68-21b09b…",935023330069,591532,"""mumbai""",2024-10-08 11:41:02.386,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22216,"""{'[amplitude] session replay i…",2024-10-08 11:41:02.386,"""session_start""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null,2455,"""international""","""session & navigation""",11,11,11,11,11,11
"""ea0a4143-2380-48e7-a83e-8ba854…",935023330069,591532,"""mumbai""",2024-10-08 11:41:03.500,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22217,"""{'rowmodel': 'server', '[ampli…",2024-10-08 11:41:03.500,""":all-accounts:configurable-tab…","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null,2455,"""international""","""account & policy management""",11,11,11,11,11,11
"""4d89977d-4734-450c-afa1-07e326…",935023330069,591532,"""mumbai""",2024-10-08 11:41:03.527,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22217,"""{'displayname': 'all policies …",2024-10-08 11:41:03.527,""":all-accounts:widget:render""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null,2455,"""international""","""account & policy management""",11,11,11,11,11,11
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""102ec3ca-a467-41a9-aa19-94f5b0…",857540480084,591532,"""nashville""",2024-05-22 09:36:09.888,20

In [15]:
# Extract relative time between start and end of the same evetns
df = df.with_columns([
    (pl.col("client_upload_time") - pl.col("client_event_time")).dt.total_seconds().alias("time_to_server"),
    (pl.col("server_upload_time") - pl.col("server_received_time")).dt.total_seconds().alias("server_to_process"),
    (pl.col("processed_time") - pl.col("server_upload_time")).dt.total_seconds().alias("processing_time")
])
df

$insert_id,amplitude_id,app,city,client_event_time,client_upload_time,country,device_family,device_id,dma,event_id,event_properties,event_time,event_type,language,library,os_name,processed_time,region,server_received_time,server_upload_time,session_id,user_id,user_properties,roles,isInternalUser,referrer,slug,session_seconds,region_grouped,event_category,client_event_hour,client_upload_hour,event_hour,processed_hour,server_received_hour,server_upload_hour,time_to_server,server_to_process,processing_time
str,i64,i64,str,datetime[μs],datetime[μs],str,str,str,str,i64,str,datetime[μs],str,str,str,str,datetime[μs],str,datetime[μs],datetime[μs],i64,str,str,str,str,str,str,i64,str,str,i8,i8,i8,i8,i8,i8,i64,i64,i64
"""251db963-6623-448e-8665-f542b8…",935023330069,591532,"""mumbai""",2024-10-08 11:41:02.385,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22216,"""{}""",2024-10-08 11:41:02.385,"""session_start""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662385,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null,0,"""international""","""session & navigation""",11,11,11,11,11,11,2,0,0
"""3a95cdac-174c-4002-8e84-8aeba8…",935023330069,591532,"""mumbai""",2024-10-08 11:41:02.386,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22215,"""{}""",2024-10-08 11:41:02.386,"""session_end""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662385,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null,0,"""international""","""session & navigation""",11,11,11,11,11,11,2,0,0
"""50dbfad3-8e24-456a-ae68-21b09b…",935023330069,591532,"""mumbai""",2024-10-08 11:41:02.386,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22216,"""{'[amplitude] session replay i…",2024-10-08 11:41:02.386,"""session_start""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null,2455,"""international""","""session & navigation""",11,11,11,11,11,11,2,0,0
"""ea0a4143-2380-48e7-a83e-8ba854…",935023330069,591532,"""mumbai""",2024-10-08 11:41:03.500,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22217,"""{'rowmodel': 'server', '[ampli…",2024-10-08 11:41:03.500,""":all-accounts:configurable-tab…","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null,2455,"""international""","""account & policy management""",11,11,11,11,11,11,1,0,0
"""4d89977d-4734-450c-afa1-07e326…",935023330069,591532,"""mumbai""",2024-10-08 11:41:03.527,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22217,"""{'displayname': 'all policies …",2024-10-08 11:41:03.527,""":all-accounts:widget:render""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null,2455,"""international""","""account & policy management""",11,11,11,11,11,11,1,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"

### Retention

There is a lot of information that we can extract from the first user visit. In particular, we can use the time between the first visits and subsequent later visits in order to infer whether the customer has an interest in the product.

We will choose to measure retention as whether the user has returned within 7 days of the last session. This indicator will be marked on a session level.

In [16]:

# Get session start and end date of a session k
session_visit_checkpoints = (df.group_by('user_id', 'session_id')
                                .agg(pl.min('event_time').alias('session_start_date'),
                                    pl.max('event_time').alias('session_end_date')))

# Rank user's sessions by start date
session_visit_checkpoints = (session_visit_checkpoints.with_columns(pl.col('session_start_date')
                                                    .rank(method = 'ordinal')
                                                    .over('user_id')
                                                    .alias('session_rank')))

# Find wait time between sessions by taking the difference
# of the end date of session k and start date of session k + 1

session_visit_checkpoints = (session_visit_checkpoints.with_columns(
                                (pl.col('session_rank') - 1).alias('prev_session_rank')))

session_visit_checkpoints = (session_visit_checkpoints.join(
                             session_visit_checkpoints.select('user_id', 'prev_session_rank', 'session_start_date'), 
                             left_on = ['user_id', 'session_rank'], 
                             right_on = ['user_id', 'prev_session_rank'],
                             how = 'left'))


# Calculate wait time between sessions
# Then create an indicator of whether this wait time is under 7 days
session_visit_checkpoints = (session_visit_checkpoints
                             .with_columns((pl.col('session_start_date_right') 
                                            - pl.col('session_end_date'))
                                            .alias('wait_time_between_sessions').dt.total_days()))

session_visit_checkpoints = session_visit_checkpoints.with_columns(
                                pl.col('wait_time_between_sessions')
                                  .is_between(1, 28)
                                  .cast(pl.Int32)
                                  .alias('returned_within_28_days'))

session_visit_checkpoints = session_visit_checkpoints.with_columns(pl.col('returned_within_28_days').fill_null(0))

# Drop columns that are irrelevant
session_visit_checkpoints = session_visit_checkpoints.drop(['user_id', 'session_end_date', 'session_start_date', 'session_start_date_right',
                                                            'wait_time_between_sessions', 'session_rank', 'prev_session_rank'])

# Merge indicator with df
df = df.join(session_visit_checkpoints, on = 'session_id', how = 'left')
df

$insert_id,amplitude_id,app,city,client_event_time,client_upload_time,country,device_family,device_id,dma,event_id,event_properties,event_time,event_type,language,library,os_name,processed_time,region,server_received_time,server_upload_time,session_id,user_id,user_properties,roles,isInternalUser,referrer,slug,session_seconds,region_grouped,event_category,client_event_hour,client_upload_hour,event_hour,processed_hour,server_received_hour,server_upload_hour,time_to_server,server_to_process,processing_time,returned_within_28_days
str,i64,i64,str,datetime[μs],datetime[μs],str,str,str,str,i64,str,datetime[μs],str,str,str,str,datetime[μs],str,datetime[μs],datetime[μs],i64,str,str,str,str,str,str,i64,str,str,i8,i8,i8,i8,i8,i8,i64,i64,i64,i32
"""251db963-6623-448e-8665-f542b8…",935023330069,591532,"""mumbai""",2024-10-08 11:41:02.385,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22216,"""{}""",2024-10-08 11:41:02.385,"""session_start""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662385,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null,0,"""international""","""session & navigation""",11,11,11,11,11,11,2,0,0,0
"""3a95cdac-174c-4002-8e84-8aeba8…",935023330069,591532,"""mumbai""",2024-10-08 11:41:02.386,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22215,"""{}""",2024-10-08 11:41:02.386,"""session_end""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662385,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null,0,"""international""","""session & navigation""",11,11,11,11,11,11,2,0,0,0
"""50dbfad3-8e24-456a-ae68-21b09b…",935023330069,591532,"""mumbai""",2024-10-08 11:41:02.386,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22216,"""{'[amplitude] session replay i…",2024-10-08 11:41:02.386,"""session_start""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null,2455,"""international""","""session & navigation""",11,11,11,11,11,11,2,0,0,0
"""ea0a4143-2380-48e7-a83e-8ba854…",935023330069,591532,"""mumbai""",2024-10-08 11:41:03.500,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22217,"""{'rowmodel': 'server', '[ampli…",2024-10-08 11:41:03.500,""":all-accounts:configurable-tab…","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null,2455,"""international""","""account & policy management""",11,11,11,11,11,11,1,0,0,0
"""4d89977d-4734-450c-afa1-07e326…",935023330069,591532,"""mumbai""",2024-10-08 11:41:03.527,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22217,"""{'displayname': 'all policies …",2024-10-08 11:41:03.527,""":all-accounts:widget:render""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null,2455,"""international""","""account & policy management""",11,11,11,11,11,11,1,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…

### Extract List-valued Columns

Next, we will extract values from list-valued columns, which can contain multiple values.

We will perform an encoding where each new indicator column would be a value from the list, and the value would represent whether the value is in that list.

Note that we will choose values from the list that are relevant to our modelling later on.

Some of the list-valued columns that we will process are `roles` and `referrer_user`

In [17]:
# Extract roles from roles, which can contain multiple roles
df = df.with_columns([
        pl.col('roles').str.contains('underwriter').alias('uw').cast(pl.Int8),
        pl.col('roles').str.contains('admin').alias('admin').cast(pl.Int8),
        pl.col('roles').str.contains('manager').alias('manager').cast(pl.Int8),
        pl.col('roles').str.contains('broker').alias('broker').cast(pl.Int8),
    ])
df

$insert_id,amplitude_id,app,city,client_event_time,client_upload_time,country,device_family,device_id,dma,event_id,event_properties,event_time,event_type,language,library,os_name,processed_time,region,server_received_time,server_upload_time,session_id,user_id,user_properties,roles,isInternalUser,referrer,slug,session_seconds,region_grouped,event_category,client_event_hour,client_upload_hour,event_hour,processed_hour,server_received_hour,server_upload_hour,time_to_server,server_to_process,processing_time,returned_within_28_days,uw,admin,manager,broker
str,i64,i64,str,datetime[μs],datetime[μs],str,str,str,str,i64,str,datetime[μs],str,str,str,str,datetime[μs],str,datetime[μs],datetime[μs],i64,str,str,str,str,str,str,i64,str,str,i8,i8,i8,i8,i8,i8,i64,i64,i64,i32,i8,i8,i8,i8
"""251db963-6623-448e-8665-f542b8…",935023330069,591532,"""mumbai""",2024-10-08 11:41:02.385,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22216,"""{}""",2024-10-08 11:41:02.385,"""session_start""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662385,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null,0,"""international""","""session & navigation""",11,11,11,11,11,11,2,0,0,0,1,0,0,0
"""3a95cdac-174c-4002-8e84-8aeba8…",935023330069,591532,"""mumbai""",2024-10-08 11:41:02.386,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22215,"""{}""",2024-10-08 11:41:02.386,"""session_end""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662385,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null,0,"""international""","""session & navigation""",11,11,11,11,11,11,2,0,0,0,1,0,0,0
"""50dbfad3-8e24-456a-ae68-21b09b…",935023330069,591532,"""mumbai""",2024-10-08 11:41:02.386,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22216,"""{'[amplitude] session replay i…",2024-10-08 11:41:02.386,"""session_start""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null,2455,"""international""","""session & navigation""",11,11,11,11,11,11,2,0,0,0,1,0,0,0
"""ea0a4143-2380-48e7-a83e-8ba854…",935023330069,591532,"""mumbai""",2024-10-08 11:41:03.500,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22217,"""{'rowmodel': 'server', '[ampli…",2024-10-08 11:41:03.500,""":all-accounts:configurable-tab…","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null,2455,"""international""","""account & policy management""",11,11,11,11,11,11,1,0,0,0,1,0,0,0
"""4d89977d-4734-450c-afa1-07e326…",935023330069,591532,"""mumbai""",2024-10-08 11:41:03.527,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22217,"""{'displayname': 'all policies …",2024-10-08 11:41:03.527,""":all-accounts:widget:render""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null,2455,"""international""","""account & policy management""",1

In [18]:
# Extract brand from referrer, which can contain multiple referrers
df = df.with_columns([
        pl.col('referrer').str.contains('google').alias('google').cast(pl.Int32),
        pl.col('referrer').str.contains('microsoft|teams|office').alias('microsoft').cast(pl.Int32),
        pl.col('referrer').str.contains('federato').alias('federato').cast(pl.Int32),
        pl.col('referrer').str.contains('portal').alias('uw-portal').cast(pl.Int32),
    ])
df

$insert_id,amplitude_id,app,city,client_event_time,client_upload_time,country,device_family,device_id,dma,event_id,event_properties,event_time,event_type,language,library,os_name,processed_time,region,server_received_time,server_upload_time,session_id,user_id,user_properties,roles,isInternalUser,referrer,slug,session_seconds,region_grouped,event_category,client_event_hour,client_upload_hour,event_hour,processed_hour,server_received_hour,server_upload_hour,time_to_server,server_to_process,processing_time,returned_within_28_days,uw,admin,manager,broker,google,microsoft,federato,uw-portal
str,i64,i64,str,datetime[μs],datetime[μs],str,str,str,str,i64,str,datetime[μs],str,str,str,str,datetime[μs],str,datetime[μs],datetime[μs],i64,str,str,str,str,str,str,i64,str,str,i8,i8,i8,i8,i8,i8,i64,i64,i64,i32,i8,i8,i8,i8,i32,i32,i32,i32
"""251db963-6623-448e-8665-f542b8…",935023330069,591532,"""mumbai""",2024-10-08 11:41:02.385,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22216,"""{}""",2024-10-08 11:41:02.385,"""session_start""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662385,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null,0,"""international""","""session & navigation""",11,11,11,11,11,11,2,0,0,0,1,0,0,0,1,0,0,0
"""3a95cdac-174c-4002-8e84-8aeba8…",935023330069,591532,"""mumbai""",2024-10-08 11:41:02.386,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22215,"""{}""",2024-10-08 11:41:02.386,"""session_end""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662385,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null,0,"""international""","""session & navigation""",11,11,11,11,11,11,2,0,0,0,1,0,0,0,1,0,0,0
"""50dbfad3-8e24-456a-ae68-21b09b…",935023330069,591532,"""mumbai""",2024-10-08 11:41:02.386,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22216,"""{'[amplitude] session replay i…",2024-10-08 11:41:02.386,"""session_start""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null,2455,"""international""","""session & navigation""",11,11,11,11,11,11,2,0,0,0,1,0,0,0,1,0,0,0
"""ea0a4143-2380-48e7-a83e-8ba854…",935023330069,591532,"""mumbai""",2024-10-08 11:41:03.500,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22217,"""{'rowmodel': 'server', '[ampli…",2024-10-08 11:41:03.500,""":all-accounts:configurable-tab…","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accounts.google.com/""",null,2455,"""international""","""account & policy management""",11,11,11,11,11,11,1,0,0,0,1,0,0,0,1,0,0,0
"""4d89977d-4734-450c-afa1-07e326…",935023330069,591532,"""mumbai""",2024-10-08 11:41:03.527,2024-10-08 11:41:04.857,"""india""","""windows""","""8bd8b6ab-370f-4b56-b38f-ad221c…",null,22217,"""{'displayname': 'all policies …",2024-10-08 11:41:03.527,""":all-accounts:widget:render""","""english""","""amplitude-ts/2.7.2""","""chrome""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""{'roles': ['underwriter'], 'tr…","""underwriter""","""false""","""https://accou

### Drop Pre-aggregate columns

Having done most of our aggregation and feature engineering, we can now drop them from our data

In [19]:
# Drop pre-aggregate columns
df = drop_columns(df, config, 1)
df

client_event_time,client_upload_time,country,device_family,dma,event_time,event_type,library,processed_time,region,server_received_time,server_upload_time,session_id,user_id,roles,isInternalUser,referrer,slug,session_seconds,region_grouped,event_category,client_event_hour,client_upload_hour,event_hour,processed_hour,server_received_hour,server_upload_hour,time_to_server,server_to_process,processing_time,returned_within_28_days,uw,admin,manager,broker,google,microsoft,federato,uw-portal
datetime[μs],datetime[μs],str,str,str,datetime[μs],str,str,datetime[μs],str,datetime[μs],datetime[μs],i64,str,str,str,str,str,i64,str,str,i8,i8,i8,i8,i8,i8,i64,i64,i64,i32,i8,i8,i8,i8,i32,i32,i32,i32
2024-10-08 11:41:02.385,2024-10-08 11:41:04.857,"""india""","""windows""",null,2024-10-08 11:41:02.385,"""session_start""","""amplitude-ts/2.7.2""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662385,"""8038ea38-8ddf-4a1a-825d-8287c0…","""underwriter""","""false""","""https://accounts.google.com/""",null,0,"""international""","""session & navigation""",11,11,11,11,11,11,2,0,0,0,1,0,0,0,1,0,0,0
2024-10-08 11:41:02.386,2024-10-08 11:41:04.857,"""india""","""windows""",null,2024-10-08 11:41:02.386,"""session_end""","""amplitude-ts/2.7.2""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662385,"""8038ea38-8ddf-4a1a-825d-8287c0…","""underwriter""","""false""","""https://accounts.google.com/""",null,0,"""international""","""session & navigation""",11,11,11,11,11,11,2,0,0,0,1,0,0,0,1,0,0,0
2024-10-08 11:41:02.386,2024-10-08 11:41:04.857,"""india""","""windows""",null,2024-10-08 11:41:02.386,"""session_start""","""amplitude-ts/2.7.2""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""underwriter""","""false""","""https://accounts.google.com/""",null,2455,"""international""","""session & navigation""",11,11,11,11,11,11,2,0,0,0,1,0,0,0,1,0,0,0
2024-10-08 11:41:03.500,2024-10-08 11:41:04.857,"""india""","""windows""",null,2024-10-08 11:41:03.500,""":all-accounts:configurable-tab…","""amplitude-ts/2.7.2""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""underwriter""","""false""","""https://accounts.google.com/""",null,2455,"""international""","""account & policy management""",11,11,11,11,11,11,1,0,0,0,1,0,0,0,1,0,0,0
2024-10-08 11:41:03.527,2024-10-08 11:41:04.857,"""india""","""windows""",null,2024-10-08 11:41:03.527,""":all-accounts:widget:render""","""amplitude-ts/2.7.2""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""underwriter""","""false""","""https://accounts.google.com/""",null,2455,"""international""","""account & policy management""",11,11,11,11,11,11,1,0,0,0,1,0,0,0,1,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024-05-22 09:36:09.888,2024-05-22 09:36:10.832,"""united states""","""windows""","""nashville, tn""",2024-05-22 09:36:09.888,"""account-lines::widget:render""","""amplitude-ts/1.8.0""",2024-05-22 09:36:11.662,"""tennessee""",2024-05-22 09:36:10.832,2024-05-22 09:36:10.833,1716368557821,"""9aecc15d-64a4-4190-80f5-b5b842…","""underwriter""","""false""",null,"""one-drive-link""",2371,"""south""","""account & policy management""",9,9,9,9,9,9,0,0,0,0,1,0,0,0,null,null,null,null
2024-05-22 09:36:09.917,2024-05-22 09:36:10.832,"""united states""","""windows""","""nashville, tn""",2024-05-22 09:36:09.917,"""account-lines::configurable-ta…","""amplitude-ts/1.8.0""",2024-05-22 09:36:11.662,"""tennessee""",2024-05-22 09:36:10.832,2024-05-22 09:36:10.833,1716368557821,"""9aecc15d-64a4-4190-80f5-b5b842…","""underwriter""","""false""",null,"""property-locations""",2371,"""south""","""account & policy management""",9,9,9,9,9,9,0,0,0,0,

### Apply One-Hot Encoding

For the rest of our categorical columns with low dimensionality, we will one-hot encode them using our preprocessing functions

In [20]:
df = encode_categorical(df, config)
df

client_event_time,client_upload_time,country,device_family_android,device_family_apple ipad,device_family_apple iphone,device_family_chrome os,device_family_chromium os,device_family_google nexus phone,device_family_ios,device_family_k,device_family_linux,device_family_mac,device_family_mac os x,device_family_ubuntu,device_family_windows,dma,event_time,event_type,library,processed_time,region,server_received_time,server_upload_time,session_id,user_id,roles,isInternalUser,referrer,slug,session_seconds,region_grouped_international,region_grouped_midwest,region_grouped_northeast,region_grouped_south,region_grouped_west,event_category_account & policy management,event_category_action center & workflow,event_category_dashboard & ui interactions,event_category_document & report interactions,event_category_filtering & searching,event_category_null,event_category_other/system events,event_category_session & navigation,event_category_submission & forms,client_event_hour,client_upload_hour,event_hour,processed_hour,server_received_hour,server_upload_hour,time_to_server,server_to_process,processing_time,returned_within_28_days,uw,admin,manager,broker,google,microsoft,federato,uw-portal
datetime[μs],datetime[μs],str,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,str,datetime[μs],str,str,datetime[μs],str,datetime[μs],datetime[μs],i64,str,str,str,str,str,i64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i8,i8,i8,i8,i8,i8,i64,i64,i64,i32,i8,i8,i8,i8,i32,i32,i32,i32
2024-10-08 11:41:02.385,2024-10-08 11:41:04.857,"""india""",0,0,0,0,0,0,0,0,0,0,0,0,1,null,2024-10-08 11:41:02.385,"""session_start""","""amplitude-ts/2.7.2""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662385,"""8038ea38-8ddf-4a1a-825d-8287c0…","""underwriter""","""false""","""https://accounts.google.com/""",null,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,11,11,11,11,11,11,2,0,0,0,1,0,0,0,1,0,0,0
2024-10-08 11:41:02.386,2024-10-08 11:41:04.857,"""india""",0,0,0,0,0,0,0,0,0,0,0,0,1,null,2024-10-08 11:41:02.386,"""session_end""","""amplitude-ts/2.7.2""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662385,"""8038ea38-8ddf-4a1a-825d-8287c0…","""underwriter""","""false""","""https://accounts.google.com/""",null,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,11,11,11,11,11,11,2,0,0,0,1,0,0,0,1,0,0,0
2024-10-08 11:41:02.386,2024-10-08 11:41:04.857,"""india""",0,0,0,0,0,0,0,0,0,0,0,0,1,null,2024-10-08 11:41:02.386,"""session_start""","""amplitude-ts/2.7.2""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""underwriter""","""false""","""https://accounts.google.com/""",null,2455,1,0,0,0,0,0,0,0,0,0,0,0,1,0,11,11,11,11,11,11,2,0,0,0,1,0,0,0,1,0,0,0
2024-10-08 11:41:03.500,2024-10-08 11:41:04.857,"""india""",0,0,0,0,0,0,0,0,0,0,0,0,1,null,2024-10-08 11:41:03.500,""":all-accounts:configurable-tab…","""amplitude-ts/2.7.2""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""underwriter""","""false""","""https://accounts.google.com/""",null,2455,1,0,0,0,0,1,0,0,0,0,0,0,0,0,11,11,11,11,11,11,1,0,0,0,1,0,0,0,1,0,0,0
2024-10-08 11:41:03.527,2024-10-08 11:41:04.857,"""india""",0,0,0,0,0,0,0,0,0,0,0,0,1,null,2024-10-08 11:41:03.527,""":all-accounts:widget:render""","""amplitude-ts/2.7.2""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""underwriter""","""false""","""https://accounts.google.com/""",null,2455,1,0,0,0,0,1,0,0,0,0,0,0,0,0,11,11,11,11,11,11,1,0,0,0,1,0,0,0,1,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024-05-22 09:36:09.888,2024-05-22 09:36:10.832,"""united states""",0,0,0,0,0,0,0,0,0,0,0,0,1,"""nashville, tn""",2024-05-22 09:36:09.888,"""account-lines::widget:r

### LOO Encoding

In [30]:
def leave_one_out_encoding(df: pl.DataFrame, column: str, target: str) -> pl.Series:
    """
    Perform Leave-One-Out Encoding on a categorical column using Polars.

    Parameters:
    df (pl.DataFrame): Input DataFrame.
    column (str): Column name to encode.
    target (str): Target column name.

    Returns:
    pl.Series: Encoded column values.
    """
    # Compute sum and count for each category
    grouped = df.group_by(column).agg(
        pl.col(target).sum().alias("sum"),
        pl.col(target).count().alias("count")
    )
    
    # Join back to get per-row values
    df = df.join(grouped, on=column, how = 'left')
    
    # Compute LOO encoding
    loo_encoding = ((df["sum"] - df[target]) / (df["count"] - 1)).round(2)
    
    df = df.with_columns(loo_encoding.alias(f"{column}_encoded"))
    df = df.with_columns(df[f"{column}_encoded"].fill_null(0))
    df = df.drop('sum', 'count')
    return df

In [32]:
df = leave_one_out_encoding(df, 'slug', 'returned_within_28_days')
df

client_event_time,client_upload_time,country,device_family_android,device_family_apple ipad,device_family_apple iphone,device_family_chrome os,device_family_chromium os,device_family_google nexus phone,device_family_ios,device_family_k,device_family_linux,device_family_mac,device_family_mac os x,device_family_ubuntu,device_family_windows,dma,event_time,event_type,library,processed_time,region,server_received_time,server_upload_time,session_id,user_id,roles,isInternalUser,referrer,slug,session_seconds,region_grouped_international,region_grouped_midwest,region_grouped_northeast,region_grouped_south,region_grouped_west,event_category_account & policy management,event_category_action center & workflow,event_category_dashboard & ui interactions,event_category_document & report interactions,event_category_filtering & searching,event_category_null,event_category_other/system events,event_category_session & navigation,event_category_submission & forms,client_event_hour,client_upload_hour,event_hour,processed_hour,server_received_hour,server_upload_hour,time_to_server,server_to_process,processing_time,returned_within_28_days,uw,admin,manager,broker,google,microsoft,federato,uw-portal,slug_encoded
datetime[μs],datetime[μs],str,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,str,datetime[μs],str,str,datetime[μs],str,datetime[μs],datetime[μs],i64,str,str,str,str,str,i64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i8,i8,i8,i8,i8,i8,i64,i64,i64,i32,i8,i8,i8,i8,i32,i32,i32,i32,f64
2024-10-08 11:41:02.385,2024-10-08 11:41:04.857,"""india""",0,0,0,0,0,0,0,0,0,0,0,0,1,null,2024-10-08 11:41:02.385,"""session_start""","""amplitude-ts/2.7.2""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662385,"""8038ea38-8ddf-4a1a-825d-8287c0…","""underwriter""","""false""","""https://accounts.google.com/""",null,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,11,11,11,11,11,11,2,0,0,0,1,0,0,0,1,0,0,0,0.0
2024-10-08 11:41:02.386,2024-10-08 11:41:04.857,"""india""",0,0,0,0,0,0,0,0,0,0,0,0,1,null,2024-10-08 11:41:02.386,"""session_end""","""amplitude-ts/2.7.2""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662385,"""8038ea38-8ddf-4a1a-825d-8287c0…","""underwriter""","""false""","""https://accounts.google.com/""",null,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,11,11,11,11,11,11,2,0,0,0,1,0,0,0,1,0,0,0,0.0
2024-10-08 11:41:02.386,2024-10-08 11:41:04.857,"""india""",0,0,0,0,0,0,0,0,0,0,0,0,1,null,2024-10-08 11:41:02.386,"""session_start""","""amplitude-ts/2.7.2""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""underwriter""","""false""","""https://accounts.google.com/""",null,2455,1,0,0,0,0,0,0,0,0,0,0,0,1,0,11,11,11,11,11,11,2,0,0,0,1,0,0,0,1,0,0,0,0.0
2024-10-08 11:41:03.500,2024-10-08 11:41:04.857,"""india""",0,0,0,0,0,0,0,0,0,0,0,0,1,null,2024-10-08 11:41:03.500,""":all-accounts:configurable-tab…","""amplitude-ts/2.7.2""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""underwriter""","""false""","""https://accounts.google.com/""",null,2455,1,0,0,0,0,1,0,0,0,0,0,0,0,0,11,11,11,11,11,11,1,0,0,0,1,0,0,0,1,0,0,0,0.0
2024-10-08 11:41:03.527,2024-10-08 11:41:04.857,"""india""",0,0,0,0,0,0,0,0,0,0,0,0,1,null,2024-10-08 11:41:03.527,""":all-accounts:widget:render""","""amplitude-ts/2.7.2""",2024-10-08 11:41:05.371,"""maharashtra""",2024-10-08 11:41:04.857,2024-10-08 11:41:04.859,1728387662386,"""8038ea38-8ddf-4a1a-825d-8287c0…","""underwriter""","""false""","""https://accounts.google.com/""",null,2455,1,0,0,0,0,1,0,0,0,0,0,0,0,0,11,11,11,11,11,11,1,0,0,0,1,0,0,0,1,0,0,0,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024-05-22 09:36:09.888,2024-05-22 09:36:10.832,"""united states""",0,0,0,0,0,0,0,0,0,0,0,0,1,"""nashville, tn""",2024-05-22 

### Extract Aggregated Columns

In [33]:
# Get columns
time_cols = ['client_upload_time', 'server_received_time', 'processed_time', 'server_upload_time', 'client_event_time', 'event_time']

one_hot_cols = [col for col in df.columns if set(df[col].drop_nulls().to_list()) <= {0, 1}]

numeric_cols = df.select(cs.numeric().exclude(one_hot_cols + ['session_id'])).columns

In [34]:
# Define aggregation scheme
agg_scheme = []

# For user_id, take the first session, assuming 1 user per session
agg_scheme.append(pl.first('user_id').name.suffix('_first'))

# Get max for all time columns
agg_scheme.extend([pl.max(time_col).name.suffix('_max') for time_col in time_cols])

# Get max for all one-hot cols, that is, keep 1 when we see it
agg_scheme.extend([pl.max(c).name.suffix('_max') for c in one_hot_cols])

# Use mean for other numeric columns
agg_scheme.extend([pl.mean(c).name.suffix('_mean') for c in numeric_cols])

In [35]:
# Aggregate data
df_session = df.group_by('session_id').agg(agg_scheme)
df_session

session_id,user_id_first,client_upload_time_max,server_received_time_max,processed_time_max,server_upload_time_max,client_event_time_max,event_time_max,device_family_android_max,device_family_apple ipad_max,device_family_apple iphone_max,device_family_chrome os_max,device_family_chromium os_max,device_family_google nexus phone_max,device_family_ios_max,device_family_k_max,device_family_linux_max,device_family_mac_max,device_family_mac os x_max,device_family_ubuntu_max,device_family_windows_max,region_grouped_international_max,region_grouped_midwest_max,region_grouped_northeast_max,region_grouped_south_max,region_grouped_west_max,event_category_account & policy management_max,event_category_action center & workflow_max,event_category_dashboard & ui interactions_max,event_category_document & report interactions_max,event_category_filtering & searching_max,event_category_null_max,event_category_other/system events_max,event_category_session & navigation_max,event_category_submission & forms_max,returned_within_28_days_max,uw_max,admin_max,manager_max,broker_max,google_max,microsoft_max,federato_max,uw-portal_max,session_seconds_mean,client_event_hour_mean,client_upload_hour_mean,event_hour_mean,processed_hour_mean,server_received_hour_mean,server_upload_hour_mean,time_to_server_mean,server_to_process_mean,processing_time_mean,slug_encoded_mean
i64,str,datetime[μs],datetime[μs],datetime[μs],datetime[μs],datetime[μs],datetime[μs],u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i32,i8,i8,i8,i8,i32,i32,i32,i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1738009295198,"""acb60a19-d8c0-4924-88c8-78ccda…",2025-01-28 12:44:37.671,2025-01-28 12:44:37.671,2025-01-28 12:44:38.655,2025-01-28 12:44:37.673,2025-01-27 20:37:12.667,2025-01-27 20:37:12.667,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,null,null,null,null,937.0,20.0,19.851852,20.0,19.851852,19.851852,19.851852,1076.148148,0.0,3.592593,0.044815
1735916130662,"""6cd4ff43-3226-4bbf-8203-5d0147…",2025-01-03 16:42:03.609,2025-01-03 16:42:03.609,2025-01-03 16:42:04.588,2025-01-03 16:42:03.611,2025-01-03 14:55:30.965,2025-01-03 14:55:30.965,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,null,null,null,null,0.0,14.0,14.5,14.0,14.5,14.5,14.5,1598.75,0.0,1.5,0.045
1714423842058,"""bbf4e34e-e76a-46e3-a5c4-cfa9c5…",2024-04-29 20:51:01.483,2024-04-29 20:51:01.483,2024-04-29 20:51:01.741,2024-04-29 20:51:01.488,2024-04-29 20:50:58.337,2024-04-29 20:50:58.337,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,null,null,null,null,16.0,20.0,20.0,20.0,20.0,20.0,20.0,2.0,0.0,0.6,0.0
1722269362779,"""94c518d5-37d3-4c2d-8c2b-72e9fb…",2024-07-29 16:09:24.368,2024-07-29 16:09:24.368,2024-07-29 16:09:25.239,2024-07-29 16:09:24.369,2024-07-29 16:09:22.780,2024-07-29 16:09:22.780,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,null,null,null,null,0.0,16.0,16.0,16.0,16.0,16.0,16.0,1.0,0.0,0.0,0.0
1696438324976,"""f8494058-23b9-466c-a10b-f201a1…",2024-03-23 17:52:40.837,2024-03-23 17:52:40.837,2024-03-23 17:52:51.412,2024-03-23 17:52:40.840,2023-10-04 16:52:11.861,2023-10-04 16:52:11.861,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,null,null,null,null,6.0,16.0,17.0,16.0,17.0,17.0,17.0,1.4778e7,0.0,10.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1715645444982,"""afe99d2f-4fce-4584-a360-967b87…",2024-05-14 00:11:13.409,2024-05-14 00:11:13.409,2024-05-14 00:11:15.431,2024-05-14 00:11:13.412,2024-05-14 00:11:11.991,2024-05-14 00:11:11.991,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5,0.0,2.0,0.0
1711108354862,"""f0457050-c716-4f31-8755-f256a9…",2024-04-01 01:45:47.954,2024-04-01 01:45:47.954,2024-04-01 01:48:17.999,2024-04-01 01:45:47.959,2024-03-22 12:01:46.294,2024-03-22 12:01:46.294,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,551.0,11.25,1.0

### Drop Remaining Columns

In [36]:
# Drop remaining columns
df_session = drop_columns(df_session, config, 2)
df_session

session_id,user_id_first,device_family_linux_max,device_family_mac os x_max,device_family_windows_max,region_grouped_international_max,region_grouped_midwest_max,region_grouped_northeast_max,region_grouped_south_max,region_grouped_west_max,event_category_account & policy management_max,event_category_action center & workflow_max,event_category_dashboard & ui interactions_max,event_category_other/system events_max,event_category_session & navigation_max,event_category_submission & forms_max,returned_within_28_days_max,uw_max,admin_max,manager_max,broker_max,google_max,microsoft_max,session_seconds_mean,client_event_hour_mean,client_upload_hour_mean,event_hour_mean,server_received_hour_mean,server_upload_hour_mean,time_to_server_mean,server_to_process_mean,processing_time_mean,slug_encoded_mean
i64,str,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i32,i8,i8,i8,i8,i32,i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1738009295198,"""acb60a19-d8c0-4924-88c8-78ccda…",0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,null,null,937.0,20.0,19.851852,20.0,19.851852,19.851852,1076.148148,0.0,3.592593,0.044815
1735916130662,"""6cd4ff43-3226-4bbf-8203-5d0147…",0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,null,null,0.0,14.0,14.5,14.0,14.5,14.5,1598.75,0.0,1.5,0.045
1714423842058,"""bbf4e34e-e76a-46e3-a5c4-cfa9c5…",0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,null,null,16.0,20.0,20.0,20.0,20.0,20.0,2.0,0.0,0.6,0.0
1722269362779,"""94c518d5-37d3-4c2d-8c2b-72e9fb…",0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,null,null,0.0,16.0,16.0,16.0,16.0,16.0,1.0,0.0,0.0,0.0
1696438324976,"""f8494058-23b9-466c-a10b-f201a1…",0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,null,null,6.0,16.0,17.0,16.0,17.0,17.0,1.4778e7,0.0,10.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1715645444982,"""afe99d2f-4fce-4584-a360-967b87…",1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.5,0.0,2.0,0.0
1711108354862,"""f0457050-c716-4f31-8755-f256a9…",0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,551.0,11.25,1.0,11.25,1.0,1.0,827437.75,0.0,150.75,0.0
1726687903267,"""cd3715bf-0c84-41f2-aa54-345093…",0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,null,null,33.0,19.0,18.666667,19.0,18.666667,18.666667,3671.0,0.0,1.0,0.02


### Impute One-Hot Columns

For the rest of the one-hot valued columns, we will fill it all with zero

In [37]:
# Impute one-hot columns
columns_to_fill = ['uw_max', 'admin_max', 'manager_max', 
                   'broker_max', 'google_max', 'microsoft_max']
df_session = df_session.with_columns(pl.col(c).fill_null(0) for c in columns_to_fill)
df_session

session_id,user_id_first,device_family_linux_max,device_family_mac os x_max,device_family_windows_max,region_grouped_international_max,region_grouped_midwest_max,region_grouped_northeast_max,region_grouped_south_max,region_grouped_west_max,event_category_account & policy management_max,event_category_action center & workflow_max,event_category_dashboard & ui interactions_max,event_category_other/system events_max,event_category_session & navigation_max,event_category_submission & forms_max,returned_within_28_days_max,uw_max,admin_max,manager_max,broker_max,google_max,microsoft_max,session_seconds_mean,client_event_hour_mean,client_upload_hour_mean,event_hour_mean,server_received_hour_mean,server_upload_hour_mean,time_to_server_mean,server_to_process_mean,processing_time_mean,slug_encoded_mean
i64,str,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i32,i8,i8,i8,i8,i32,i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1738009295198,"""acb60a19-d8c0-4924-88c8-78ccda…",0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,937.0,20.0,19.851852,20.0,19.851852,19.851852,1076.148148,0.0,3.592593,0.044815
1735916130662,"""6cd4ff43-3226-4bbf-8203-5d0147…",0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0.0,14.0,14.5,14.0,14.5,14.5,1598.75,0.0,1.5,0.045
1714423842058,"""bbf4e34e-e76a-46e3-a5c4-cfa9c5…",0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,16.0,20.0,20.0,20.0,20.0,20.0,2.0,0.0,0.6,0.0
1722269362779,"""94c518d5-37d3-4c2d-8c2b-72e9fb…",0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0.0,16.0,16.0,16.0,16.0,16.0,1.0,0.0,0.0,0.0
1696438324976,"""f8494058-23b9-466c-a10b-f201a1…",0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,6.0,16.0,17.0,16.0,17.0,17.0,1.4778e7,0.0,10.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1715645444982,"""afe99d2f-4fce-4584-a360-967b87…",1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.5,0.0,2.0,0.0
1711108354862,"""f0457050-c716-4f31-8755-f256a9…",0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,551.0,11.25,1.0,11.25,1.0,1.0,827437.75,0.0,150.75,0.0
1726687903267,"""cd3715bf-0c84-41f2-aa54-345093…",0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,33.0,19.0,18.666667,19.0,18.666667,18.666667,3671.0,0.0,1.0,0.02


In [38]:
# export parquet
df_session.write_parquet(config['pipeline']['output_data'])